In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import random
import pandas as pd
from sklearn.cross_validation import train_test_split

In [2]:
image_size = [224,224]
num_classes = 10

drop_out = 0.2
learning_rate = 0.0005
epochs = 1
batch_size = 64
display_step = 1#float('inf')

In [3]:
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [4]:
b = np.zeros([len(y_train),num_classes])
for index, y in enumerate(y_train):
    b[index,y] = 1
y_train = b
b = np.zeros([len(y_test),num_classes])
for index, y in enumerate(y_test):
    b[index,y] = 1
y_test = b

In [5]:
X_TF = tf.placeholder(tf.float32,[None,image_size[0],image_size[1],3])
Y_TF = tf.placeholder(tf.float32, [None, num_classes])

In [ ]:
# Download and use NASNet feature vector module.
module = hub.Module("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/1",trainable=True)

with tf.name_scope("module") as scope:
    features = module(X_TF)
with tf.name_scope("dense") as scope:
    logits = tf.layers.dense(inputs=features,units=num_classes)

In [7]:
with tf.name_scope("cost") as scope:
    cost = tf.losses.softmax_cross_entropy(Y_TF,logits)
cost_summ = tf.summary.scalar("cost",cost)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y_TF, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [8]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter("/tmp/summaries/CIRAF10",sess.graph)
    
    test_images = x_test[:batch_size]
    test_labels = y_test[:batch_size]
    test_images = tf.image.resize_images(test_images,[image_size[0],image_size[1]])
    test_images = sess.run(test_images)
    
    for epoch in range(epochs):
        num_steps = int(len(x_train)/batch_size)
        for step in range(num_steps):         
            batch_xs = x_train[batch_size*step:batch_size*(step+1)]
            batch_ys = y_train[batch_size*step:batch_size*(step+1)]
            batch_xs = tf.image.resize_images(batch_xs,[image_size[0],image_size[1]])
            batch_xs = sess.run(batch_xs)
            
            #print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
            #print(len(sess.run(tf.global_variables())))
            #print(sess.run(tf.global_variables())[551])
            #print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
            
            summary,_ = sess.run([merged,optimizer], feed_dict={X_TF:batch_xs,Y_TF:batch_ys})
            writer.add_summary(summary,step+(epoch*num_steps))
            if step % display_step == 0:
                print("step:",step,"- epoch:",epoch)
                test_pred = sess.run(logits,feed_dict={X_TF: test_images})
                print("y_hat:",np.argmax(test_pred[:5],axis=1))
                print("y    :",np.argmax(test_labels[:5],axis=1))
                print("train_cost:",sess.run(cost,feed_dict={X_TF:batch_xs,Y_TF:batch_ys}))
                print("valid_cost:",sess.run(cost,feed_dict={X_TF:test_images,Y_TF:test_labels}))
                print("Accuracy:",sess.run(accuracy, feed_dict={X_TF:test_images,Y_TF:test_labels}))
                print("________________________________________")

    print()
    print()
    print("Optimization Finished!")
    print("Test accuracy:",sess.run(accuracy, feed_dict={X_TF:test_images,Y_TF:test_labels}))

step: 0 - epoch: 0
y_hat: [1 1 1 1 1]
y    : [3 8 8 0 6]
train_cost: 3612.1838
valid_cost: 4329.664
Accuracy: 0.046875
________________________________________
step: 1 - epoch: 0
y_hat: [9 9 9 9 9]
y    : [3 8 8 0 6]
train_cost: 2980.1133
valid_cost: 2464.8335
Accuracy: 0.140625
________________________________________
step: 2 - epoch: 0
y_hat: [3 3 3 3 3]
y    : [3 8 8 0 6]
train_cost: 1182.0847
valid_cost: 1006.19147
Accuracy: 0.09375
________________________________________
step: 3 - epoch: 0
y_hat: [0 0 0 0 0]
y    : [3 8 8 0 6]
train_cost: 2937.3745
valid_cost: 2702.229
Accuracy: 0.09375
________________________________________
step: 4 - epoch: 0
y_hat: [2 2 2 2 2]
y    : [3 8 8 0 6]
train_cost: 839.59033
valid_cost: 778.7406
Accuracy: 0.03125
________________________________________
step: 5 - epoch: 0
y_hat: [5 5 5 5 5]
y    : [3 8 8 0 6]
train_cost: 543.1626
valid_cost: 455.23853
Accuracy: 0.109375
________________________________________
step: 6 - epoch: 0
y_hat: [7 7 7 7 7]
y 

train_cost: 1.9013865
valid_cost: 1.9398035
Accuracy: 0.359375
________________________________________
step: 52 - epoch: 0
y_hat: [3 9 9 0 4]
y    : [3 8 8 0 6]
train_cost: 1.9527643
valid_cost: 1.9323726
Accuracy: 0.203125
________________________________________
step: 53 - epoch: 0
y_hat: [3 9 0 0 5]
y    : [3 8 8 0 6]
train_cost: 2.073287
valid_cost: 1.9507055
Accuracy: 0.21875
________________________________________
step: 54 - epoch: 0
y_hat: [3 9 8 8 2]
y    : [3 8 8 0 6]
train_cost: 1.8685882
valid_cost: 2.0246515
Accuracy: 0.1875
________________________________________
step: 55 - epoch: 0
y_hat: [3 8 8 8 4]
y    : [3 8 8 0 6]
train_cost: 1.7577145
valid_cost: 2.0672364
Accuracy: 0.234375
________________________________________
step: 56 - epoch: 0
y_hat: [3 9 8 8 4]
y    : [3 8 8 0 6]
train_cost: 1.8090527
valid_cost: 1.9007658
Accuracy: 0.21875
________________________________________
step: 57 - epoch: 0
y_hat: [3 9 9 9 4]
y    : [3 8 8 0 6]
train_cost: 1.9638915
valid_cost:

train_cost: 1.7103798
valid_cost: 1.811892
Accuracy: 0.328125
________________________________________
step: 103 - epoch: 0
y_hat: [3 8 8 8 6]
y    : [3 8 8 0 6]
train_cost: 1.9265699
valid_cost: 1.8008895
Accuracy: 0.34375
________________________________________
step: 104 - epoch: 0
y_hat: [1 1 8 1 4]
y    : [3 8 8 0 6]
train_cost: 1.6442525
valid_cost: 1.8588009
Accuracy: 0.34375
________________________________________
step: 105 - epoch: 0
y_hat: [1 1 8 1 4]
y    : [3 8 8 0 6]
train_cost: 1.5907233
valid_cost: 2.0043845
Accuracy: 0.296875
________________________________________
step: 106 - epoch: 0
y_hat: [3 8 8 8 6]
y    : [3 8 8 0 6]
train_cost: 1.7720132
valid_cost: 1.8361164
Accuracy: 0.34375
________________________________________
step: 107 - epoch: 0
y_hat: [3 8 8 8 4]
y    : [3 8 8 0 6]
train_cost: 1.938644
valid_cost: 1.7840556
Accuracy: 0.3125
________________________________________
step: 108 - epoch: 0
y_hat: [1 8 8 8 4]
y    : [3 8 8 0 6]
train_cost: 1.9249282
valid_c

step: 153 - epoch: 0
y_hat: [3 9 8 8 4]
y    : [3 8 8 0 6]
train_cost: 1.5017152
valid_cost: 1.6917613
Accuracy: 0.328125
________________________________________
step: 154 - epoch: 0
y_hat: [3 1 8 8 4]
y    : [3 8 8 0 6]
train_cost: 1.5931839
valid_cost: 1.6892385
Accuracy: 0.359375
________________________________________
step: 155 - epoch: 0
y_hat: [3 1 8 8 4]
y    : [3 8 8 0 6]
train_cost: 1.5892469
valid_cost: 1.6968503
Accuracy: 0.359375
________________________________________
step: 156 - epoch: 0
y_hat: [3 1 8 8 6]
y    : [3 8 8 0 6]
train_cost: 1.6621346
valid_cost: 1.7071681
Accuracy: 0.34375
________________________________________
step: 157 - epoch: 0
y_hat: [3 1 8 8 6]
y    : [3 8 8 0 6]
train_cost: 1.5815198
valid_cost: 1.7049891
Accuracy: 0.359375
________________________________________
step: 158 - epoch: 0
y_hat: [3 1 8 8 6]
y    : [3 8 8 0 6]
train_cost: 1.6299858
valid_cost: 1.7189844
Accuracy: 0.359375
________________________________________
step: 159 - epoch: 0
y_

Accuracy: 0.3125
________________________________________
step: 204 - epoch: 0
y_hat: [2 1 8 8 4]
y    : [3 8 8 0 6]
train_cost: 1.530017
valid_cost: 1.8010076
Accuracy: 0.328125
________________________________________
step: 205 - epoch: 0
y_hat: [3 1 8 8 6]
y    : [3 8 8 0 6]
train_cost: 1.6742408
valid_cost: 1.6678858
Accuracy: 0.40625
________________________________________
step: 206 - epoch: 0
y_hat: [3 1 8 8 6]
y    : [3 8 8 0 6]
train_cost: 1.5638161
valid_cost: 1.6099483
Accuracy: 0.40625
________________________________________
step: 207 - epoch: 0
y_hat: [3 1 8 8 6]
y    : [3 8 8 0 6]
train_cost: 1.7291923
valid_cost: 1.6340454
Accuracy: 0.40625
________________________________________
step: 208 - epoch: 0
y_hat: [3 1 8 8 6]
y    : [3 8 8 0 6]
train_cost: 1.5523549
valid_cost: 1.6539011
Accuracy: 0.390625
________________________________________
step: 209 - epoch: 0
y_hat: [2 1 8 8 4]
y    : [3 8 8 0 6]
train_cost: 1.7230273
valid_cost: 1.6661539
Accuracy: 0.390625
_________

...

step: 560 - epoch: 0
y_hat: [3 1 8 0 6]
y    : [3 8 8 0 6]
train_cost: 1.1799252
valid_cost: 1.2300162
Accuracy: 0.53125
________________________________________
step: 561 - epoch: 0
y_hat: [5 1 8 0 6]
y    : [3 8 8 0 6]
train_cost: 1.1550272
valid_cost: 1.2490377
Accuracy: 0.53125
________________________________________
step: 562 - epoch: 0
y_hat: [5 1 8 0 4]
y    : [3 8 8 0 6]
train_cost: 1.177988
valid_cost: 1.2982626
Accuracy: 0.5
________________________________________
step: 563 - epoch: 0
y_hat: [5 1 8 0 4]
y    : [3 8 8 0 6]
train_cost: 1.2833542
valid_cost: 1.3306503
Accuracy: 0.46875
________________________________________
step: 564 - epoch: 0
y_hat: [5 1 8 0 4]
y    : [3 8 8 0 6]
train_cost: 1.1916883
valid_cost: 1.3625757
Accuracy: 0.4375
________________________________________
step: 565 - epoch: 0
y_hat: [3 1 8 8 4]
y    : [3 8 8 0 6]
train_cost: 1.3469641
valid_cost: 1.3385918
Accuracy: 0.453125
________________________________________
step: 566 - epoch: 0
y_hat: [3 1 

step: 611 - epoch: 0
y_hat: [5 8 8 8 4]
y    : [3 8 8 0 6]
train_cost: 1.3147411
valid_cost: 1.179523
Accuracy: 0.578125
________________________________________
step: 612 - epoch: 0
y_hat: [5 8 8 8 3]
y    : [3 8 8 0 6]
train_cost: 1.3983293
valid_cost: 1.1516345
Accuracy: 0.578125
________________________________________
step: 613 - epoch: 0
y_hat: [3 8 8 8 6]
y    : [3 8 8 0 6]
train_cost: 1.0905588
valid_cost: 1.1248765
Accuracy: 0.65625
________________________________________
step: 614 - epoch: 0
y_hat: [3 1 8 8 6]
y    : [3 8 8 0 6]
train_cost: 1.1912432
valid_cost: 1.1444609
Accuracy: 0.578125
________________________________________
step: 615 - epoch: 0
y_hat: [3 1 8 8 6]
y    : [3 8 8 0 6]
train_cost: 1.1737121
valid_cost: 1.184002
Accuracy: 0.546875
________________________________________
step: 616 - epoch: 0
y_hat: [3 1 8 8 6]
y    : [3 8 8 0 6]
train_cost: 1.2014914
valid_cost: 1.1921507
Accuracy: 0.546875
________________________________________
step: 617 - epoch: 0
y_ha

Accuracy: 0.5625
________________________________________
step: 662 - epoch: 0
y_hat: [3 8 8 0 4]
y    : [3 8 8 0 6]
train_cost: 0.957701
valid_cost: 1.1704153
Accuracy: 0.578125
________________________________________
step: 663 - epoch: 0
y_hat: [3 8 8 0 4]
y    : [3 8 8 0 6]
train_cost: 0.9973061
valid_cost: 1.2150342
Accuracy: 0.5625
________________________________________
step: 664 - epoch: 0
y_hat: [3 8 8 8 4]
y    : [3 8 8 0 6]
train_cost: 1.1770569
valid_cost: 1.2124273
Accuracy: 0.546875
________________________________________
step: 665 - epoch: 0
y_hat: [3 8 8 8 4]
y    : [3 8 8 0 6]
train_cost: 1.2795409
valid_cost: 1.1132318
Accuracy: 0.578125
________________________________________
step: 666 - epoch: 0
y_hat: [3 8 8 8 6]
y    : [3 8 8 0 6]
train_cost: 0.9921335
valid_cost: 1.0581126
Accuracy: 0.625
________________________________________
step: 667 - epoch: 0
y_hat: [3 8 8 8 6]
y    : [3 8 8 0 6]
train_cost: 1.1861515
valid_cost: 1.0535599
Accuracy: 0.578125
___________

train_cost: 1.0128713
valid_cost: 1.3020736
Accuracy: 0.484375
________________________________________
step: 713 - epoch: 0
y_hat: [3 1 8 0 6]
y    : [3 8 8 0 6]
train_cost: 1.2787185
valid_cost: 1.3164363
Accuracy: 0.5
________________________________________
step: 714 - epoch: 0
y_hat: [3 8 8 8 6]
y    : [3 8 8 0 6]
train_cost: 1.0878707
valid_cost: 1.1569573
Accuracy: 0.59375
________________________________________
step: 715 - epoch: 0
y_hat: [3 8 8 8 6]
y    : [3 8 8 0 6]
train_cost: 1.2403353
valid_cost: 1.1746521
Accuracy: 0.578125
________________________________________
step: 716 - epoch: 0
y_hat: [5 8 8 8 6]
y    : [3 8 8 0 6]
train_cost: 1.150389
valid_cost: 1.191592
Accuracy: 0.5625
________________________________________
step: 717 - epoch: 0
y_hat: [5 8 8 8 6]
y    : [3 8 8 0 6]
train_cost: 1.2101237
valid_cost: 1.2728869
Accuracy: 0.53125
________________________________________
step: 718 - epoch: 0
y_hat: [5 8 8 8 6]
y    : [3 8 8 0 6]
train_cost: 1.2439848
valid_cost:

step: 763 - epoch: 0
y_hat: [5 1 8 8 6]
y    : [3 8 8 0 6]
train_cost: 1.0059755
valid_cost: 1.1964505
Accuracy: 0.5625
________________________________________
step: 764 - epoch: 0
y_hat: [3 1 8 0 6]
y    : [3 8 8 0 6]
train_cost: 1.0297087
valid_cost: 1.1070651
Accuracy: 0.609375
________________________________________
step: 765 - epoch: 0
y_hat: [3 1 8 0 6]
y    : [3 8 8 0 6]
train_cost: 1.2908506
valid_cost: 1.0095135
Accuracy: 0.640625
________________________________________
step: 766 - epoch: 0
y_hat: [3 1 8 0 6]
y    : [3 8 8 0 6]
train_cost: 0.85330653
valid_cost: 0.9841113
Accuracy: 0.671875
________________________________________
step: 767 - epoch: 0
y_hat: [3 1 8 8 6]
y    : [3 8 8 0 6]
train_cost: 1.1460582
valid_cost: 0.9942793
Accuracy: 0.65625
________________________________________
step: 768 - epoch: 0
y_hat: [3 8 8 8 6]
y    : [3 8 8 0 6]
train_cost: 0.97097695
valid_cost: 0.9984551
Accuracy: 0.625
________________________________________
step: 769 - epoch: 0
y_hat